In [4]:
#Imports
import os
import shutil
import pandas as pd
import sqlite3

In [5]:
#Filters out the required excel files (makes a copy)
dataset_dir = "C:\\Users\\shash\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\Data\\All Data"  # replace with the path to your dataset folder
filtered_dir = "C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2"  # replace with the path to the folder where you want to store the filtered data
if not os.path.exists(filtered_dir):
    os.makedirs(filtered_dir)

for month_folder in os.listdir(dataset_dir):
    if not month_folder.startswith("20"):  # skip any folders that don't start with a date
        continue
    for csv_file in os.listdir(os.path.join(dataset_dir, month_folder)):
        if not csv_file.endswith("-metropolitan-street.csv"):  # skip any files that don't match the desired format
            continue
        source_path = os.path.join(dataset_dir, month_folder, csv_file)
        destination_path = os.path.join(filtered_dir, csv_file)
        shutil.copy(source_path, destination_path)  # copy the matching file to the filtered_data folder


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\Data\\All Data'

In [ ]:

folder_path = "C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\Cleaned Data\\Street"  # replace with the path to your Excel files folder
df_list = []
for file in os.listdir(folder_path):
    if file.endswith(".csv"):  # only read in CSV files
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        df_list.append(df)
combined_df = pd.concat(df_list, ignore_index=True)


In [ ]:
print(combined_df)

                                                   Crime ID    Month  \
0                                                       NaN  2010-12   
1                                                       NaN  2010-12   
2                                                       NaN  2010-12   
3                                                       NaN  2010-12   
4                                                       NaN  2010-12   
...                                                     ...      ...   
12991964  c3398c086a685b13bbc2bdc8497179a4fe3e807cfe1f2e...  2023-01   
12991965  ce406aaec27adeb767c4cfb60a85c453d0e15bb08d5e90...  2023-01   
12991966  bebf046f2da32d2ec393e677443cda9fa0ebbe8c38bb02...  2023-01   
12991967  7015c23748b5935ab6367416faeff367df58354ea322e9...  2023-01   
12991968  bd86828d854760836e693a939fe97ad5a4af5abfa9dc07...  2023-01   

                          Reported by                 Falls within  Longitude  \
0         Metropolitan Police Service  Metropolitan Po

In [ ]:
output_path ="C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\Cleaned Data\\conc.csv"
combined_df.to_csv(output_path, index=False)  # save the combined DataFrame to a new CSV file

In [6]:


# Load CSV data into a pandas DataFrame
df = combined_df

# Connect to an SQLite database (creates a new database file if it doesn't exist)
conn = sqlite3.connect('C:\\Users\\shash\\OneDrive - TU Eindhoven\\Shashank Prabhu University\\Year 2\\Year 2 Q4\\Data Challenge 2\\Git DC2\\Cleaned Data\\database_conc.db')

# Write DataFrame to an SQL database table
df.to_sql('table_name', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()